In [32]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Conv3D, Flatten, MaxPool2D, AveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.python.keras import metrics
import matplotlib.pyplot as plt
from utilities.rectified_adam import RAdam
import numpy as np
import sys
from utilities.pythonDB import writeToDB, deleteExistingPrimaryKeyDB
from utilities.data_preprocessors import get_random_data, MetricsAfterEachEpoch
from keras import regularizers
from keras.constraints import unit_norm

In [48]:
global MAX_EPOCHS, MAX_BATCH_SIZE, architecture, label, keywords
MAX_EPOCHS, MAX_BATCH_SIZE = 10, 1024
architecture = 'ZFNet'
label, keywords = 'noisy_bl-experiments_test', 'benchmark_testing'

In [60]:
MAX_TRAINING = 15000
MAX_VALIDATION = 5000
MAX_TESTING = 5000

base = 'noisy_dataset/noisy/cifar_10_{}.npy'
x_train_max, y_train = np.load(base.format('x_train')), np.load(base.format('y_train'))
x_test_max, y_test = np.load(base.format('x_test')), np.load(base.format('y_test'))
# cifar = tf.keras.datasets.cifar10 
# (_, _), (x_test_max, y_test) = cifar.load_data()
y_train_max, y_test_max = to_categorical(y_train, num_classes=10), to_categorical(y_test, num_classes=10)

train_cutoff = int(x_train_max.shape[0] - MAX_VALIDATION*4)

x_train, y_train = get_random_data(x_train_max, y_train_max, 0, train_cutoff, MAX_TRAINING)
x_val, y_val = get_random_data(x_test_max, y_test_max, 0, x_test_max.shape[0], int(MAX_TESTING/2))
x_test, y_test = get_random_data(x_test_max, y_test_max, 0, x_test_max.shape[0], MAX_TESTING)

print ("Number of Training Samples: X={}, Y={}".format(x_train.shape[0], y_train.shape[0]))
print ("Number of Validation Samples: X={}, Y={}".format(x_val.shape[0], y_val.shape[0]))
print ("Number of Test Samples: X={}, Y={}".format(x_test.shape[0], y_test.shape[0]))

Number of Training Samples: X=15000, Y=15000
Number of Validation Samples: X=2500, Y=2500
Number of Test Samples: X=5000, Y=5000


In [61]:
def model_extractor(activation_func, weight_decay=5e-4):
    model = Sequential()

    #Instantiating first set of Layers
    model.add(Conv2D(32, kernel_size=(3, 3), activation=activation_func, padding='same', 
                    kernel_constraint=unit_norm(), kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size=(3, 3), activation=activation_func, padding='same', 
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2), strides=(1, 1), name='pool1'))
    model.add(Dropout(0.2))

    # Instantiating second set of Layers
    model.add(Conv2D(64, kernel_size=(3, 3), activation=activation_func, padding='same', 
                     kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3, 3), activation=activation_func, padding='same', 
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2, 2), strides=(1, 1), name='pool2'))
    model.add(Dropout(0.2))

    # Instantiating set of FCs
    model.add(Flatten())
    model.add(Dense(512, activation=activation_func)) 
    model.add(BatchNormalization())
    #Output Layer
    model.add(Dense(10, activation='softmax'))    
    return model

In [62]:
def diff_optimizer(curr_optimizer, activation_func, label=None, curr_epochs=MAX_EPOCHS):
    model = model_extractor(activation_func)
    model.compile(loss="categorical_crossentropy", optimizer=curr_optimizer, validation_data=(x_val, y_val),
                  metrics=["accuracy", "mae", "mse"])
    history = model.fit(x_train, y_train, epochs=curr_epochs, batch_size=1024).history
    score = model.evaluate(x_test, y_test, batch_size=MAX_BATCH_SIZE)
    
    #Saving the model
#     if label == 'RAdam':
#         model.save("../../saved_models_noisy/{}_{}_{}.hdf5".format('RAdam', activation_func, architecture))
#     else: 
#         model.save("../../saved_models_noisy/{}_{}_{}.hdf5".format(curr_optimizer, activation_func, architecture))
    return score, history

In [63]:
testing_optimizers = ['Adam'] #['RAdam', 'Adam', 'NAdam', 'SGD']
testing_activations = ['relu']#['tanh', 'relu', 'selu'] #'selu'


for optimizer in testing_optimizers:
    for activation in testing_activations:
        loss, accuracy, mae, mse = {}, {}, {}, {}
        train_loss_dict, train_accuracy_dict, train_mae_dict, train_mse_dict = {}, {}, {}, {}

        if optimizer == 'RAdam':
            score_returned, history = diff_optimizer(RAdam(lr=0.05), activation, label='RAdam')
        else:
            score_returned, history = diff_optimizer(optimizer, activation)

        bg = (score_returned[0], score_returned[1], score_returned[2], score_returned[3], history.get('loss'),\
          history.get('accuracy'), history.get('mae'), history.get('mae'))

        #Delete Existing Primary Keys and then write to DB
        deleteExistingPrimaryKeyDB(optimizer, activation, architecture, label, keywords)
        writeToDB(optimizer, activation, architecture, bg, label, keywords)

print ("Completed Execution for architecture={}".format(architecture))

Train on 15000 samples
Epoch 1/10
15000/15000 [==============================] - 193s 13ms/sample - loss: 2.4658 - acc: 0.3917 - mean_absolute_error: 0.1467 - mean_squared_error: 0.0806
Epoch 2/10
15000/15000 [==============================] - 187s 12ms/sample - loss: 1.6604 - acc: 0.5359 - mean_absolute_error: 0.1262 - mean_squared_error: 0.0619
Epoch 3/10
15000/15000 [==============================] - 187s 12ms/sample - loss: 1.4326 - acc: 0.5897 - mean_absolute_error: 0.1136 - mean_squared_error: 0.0549
Epoch 4/10
15000/15000 [==============================] - 189s 13ms/sample - loss: 1.2557 - acc: 0.6322 - mean_absolute_error: 0.1027 - mean_squared_error: 0.0493
Epoch 5/10
15000/15000 [==============================] - 187s 12ms/sample - loss: 1.1074 - acc: 0.6775 - mean_absolute_error: 0.0916 - mean_squared_error: 0.0438
Epoch 6/10
15000/15000 [==============================] - 201s 13ms/sample - loss: 1.0024 - acc: 0.7127 - mean_absolute_error: 0.0835 - mean_squared_error: 0.0396

In [ ]:
# Baseline - 50.02% (75.16%)%

In [ ]:
#keras.callbacks.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=False)

